# Day 13: Claw Contraption

Next up: the lobby of a resort on a tropical island. The Historians take a moment to admire the **hexagonal floor tiles** before spreading out.

Fortunately, it looks like the resort has a new **arcade**! Maybe you can win some prizes from the claw machines?

---

### The Claw Machines

The claw machines here are a little unusual:
- Instead of a joystick or directional buttons to control the claw, these machines have two buttons labeled **A** and **B**.
- Worse, you can't just put in a token and play; it costs:
  - **3 tokens** to push the A button.
  - **1 token** to push the B button.

With some experimentation, you figure out that each machine's buttons are configured to move the claw a specific amount:
- **Button A**: Moves the claw a certain number of units to the right (**X axis**) and forward (**Y axis**) with each press.
- **Button B**: Moves the claw a different number of units along the same axes.

Each machine contains **one prize**, and to win the prize:
- The claw must be positioned **exactly above the prize** on both the X and Y axes.

---

### The Challenge

You wonder: **What is the smallest number of tokens you would have to spend to win as many prizes as possible?**

You assemble a list of each machine's button behavior and prize location. For example:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
```

This list describes the button configuration and prize location for four different claw machines.

---

### Example: First Claw Machine

For the **first claw machine**:
- Pressing **A** moves the claw:
  - **94 units** along the X axis.
  - **34 units** along the Y axis.
- Pressing **B** moves the claw:
  - **22 units** along the X axis.
  - **67 units** along the Y axis.
- The prize is located at:
  - **X = 8400**.
  - **Y = 5400**.

To win the prize:
- The claw must move exactly **8400 units along the X axis** and **5400 units along the Y axis** from its initial position.

The cheapest solution is:
- Push the **A button** **80 times**.
- Push the **B button** **40 times**.

This works because:
- For the X axis: \( 80 * 94 + 40 * 22 = 8400 \).
- For the Y axis: \( 80 * 34 + 40 * 67 = 5400 \).

The total cost:
- \( 80 * 3 = 240 \) tokens for the A button.
- \( 40 * 1 = 40 \) tokens for the B button.
- **Total: 280 tokens**.

---

### Example: Other Claw Machines

- **Second Claw Machine**:
  - There is no combination of A and B presses that will align the claw with the prize.

- **Third Claw Machine**:
  - The cheapest solution:
    - **A button**: 38 presses.
    - **B button**: 86 presses.
  - Total cost: \( 38 * 3 + 86 * 1 = 200 \) tokens.

- **Fourth Claw Machine**:
  - No combination of A and B presses will align the claw with the prize.

---

### Results

For the example list:
- **Prizes won**: 2 (first and third claw machines).
- **Total tokens spent**: \( 280 + 200 = 480 \).

---

### Estimation

You estimate:
- Each button would need to be pressed no more than **100 times** to win a prize.

---

### Task

Figure out how to win as many prizes as possible.  
**What is the fewest tokens you would have to spend to win all possible prizes?**

In [1]:
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Type, TypeVar
from abc import ABC, abstractmethod
import re
import numpy as np
from scipy.optimize import minimize

# Define a generic type for machines
M = TypeVar("M", bound="Machine")


@dataclass
class Machine(ABC):
    """
    Abstract base class for all machines.
    """

    move_a: Tuple[int, int]
    move_b: Tuple[int, int]
    prize_location: Tuple[int, int]
    cost_a: int = 3
    cost_b: int = 1

    @abstractmethod
    def token_cost(self, button_presses: Tuple[int, int]) -> float:
        """
        Calculate the cost for a given number of button presses.
        """
        pass

    @abstractmethod
    def find_minimum_tokens(
        self,
        initial_guess: Tuple[int, int],
        press_bounds: List[Tuple[int, int]],
        max_iterations: int,
        disp: bool,
    ) -> Optional[Tuple[int, int]]:
        """
        Find the optimal solution for the machine.
        """
        pass


class ClawMachine(Machine):
    """
    Represents a single claw machine configuration and its prize location.
    """

    def token_cost(self, button_presses: Tuple[int, int]) -> float:
        """Calculate the total token cost."""
        return button_presses[0] * self.cost_a + button_presses[1] * self.cost_b

    def position_constraint(self, button_presses: Tuple[int, int]) -> float:
        """Ensure the claw reaches the prize location."""
        x = self.move_a[0] * button_presses[0] + self.move_b[0] * button_presses[1]
        y = self.move_a[1] * button_presses[0] + self.move_b[1] * button_presses[1]
        dx, dy = x - self.prize_location[0], y - self.prize_location[1]
        return dx**2 + dy**2

    def find_minimum_tokens(
        self,
        initial_guess: Tuple[int, int] = (1, 1),
        press_bounds: List[Tuple[int, int]] = [(0, 100), (0, 100)],
        max_iterations: int = 1000,
        disp: bool = False,
    ) -> Optional[Tuple[int, int]]:
        """Find the minimum-cost combination of button presses."""
        constraints = [{"type": "eq", "fun": self.position_constraint}]
        options = {"maxiter": max_iterations, "disp": disp}

        result = minimize(
            self.token_cost,
            initial_guess,
            method="SLSQP",
            bounds=press_bounds,
            constraints=constraints,
            options=options,
        )
        optimal_presses = np.round(result.x).astype(int)
        if np.isclose(self.position_constraint(optimal_presses), 0):
            return tuple(opt.item() for opt in optimal_presses)
        return None


@dataclass
class Arcade:
    """
    Represents an arcade containing multiple machine instances.
    """

    machines: List[Machine] = field(default_factory=list)

    @classmethod
    def from_text(cls, input_text: str, machine_cls: Type[M]) -> "Arcade":
        """
        Create an Arcade instance from the input text.

        Args:
            input_text (str): Multiline string describing machine configurations.
            machine_cls (Type[M]): The class of the machines to instantiate.

        Returns:
            Arcade: An Arcade instance containing multiple machine objects.
        """
        # Split input into chunks for each machine
        machine_blocks = input_text.strip().split("\n\n")
        machines = []

        for block in machine_blocks:
            # Extract Button A details
            move_a_match = re.search(r"Button A: X\+(-?\d+), Y\+(-?\d+)", block)
            move_a = (int(move_a_match.group(1)), int(move_a_match.group(2)))

            # Extract Button B details
            move_b_match = re.search(r"Button B: X\+(-?\d+), Y\+(-?\d+)", block)
            move_b = (int(move_b_match.group(1)), int(move_b_match.group(2)))

            # Extract Prize details
            prize_match = re.search(r"Prize: X=(-?\d+), Y=(-?\d+)", block)
            prize_location = (int(prize_match.group(1)), int(prize_match.group(2)))

            # Create a machine instance and add it to the list
            machines.append(machine_cls(move_a, move_b, prize_location))

        return cls(machines)

    @classmethod
    def from_file(cls, filepath: str, machine_cls: Type[M]) -> "Arcade":
        """
        Create an Arcade instance from a file.

        Args:
            filepath (str): Path to the file containing machine configurations.
            machine_cls (Type[M]): The class of the machines to instantiate.

        Returns:
            Arcade: An Arcade instance containing multiple machine objects.
        """
        with open(filepath, "r", encoding="utf-8") as file:
            data = file.read()
        return cls.from_text(data, machine_cls)


# Example Usage
arcade = Arcade.from_file("./example.txt", ClawMachine)

total_tokens = 0
for i, machine in enumerate(arcade.machines):
    winning_combo = machine.find_minimum_tokens()
    if winning_combo is not None:
        print(f"Press {winning_combo} buttons (a, b) to win at machine {i}")
        total_tokens += machine.token_cost(winning_combo)

print("Total tokens spent to win all prizes:", total_tokens)


Press (80, 40) buttons (a, b) to win at machine 0
Press (38, 86) buttons (a, b) to win at machine 2
Total tokens spent to win all prizes: 480


In [2]:
# Load arcade configurations from a file
arcade = Arcade.from_file("./input.txt", ClawMachine)

total_tokens = 0
for machine in arcade.machines:
    winning_combo = machine.find_minimum_tokens()
    if winning_combo is not None:
        total_tokens += machine.token_cost(winning_combo)

print("Total tokens spent to win all prizes:", total_tokens)

Total tokens spent to win all prizes: 31589


# --- Part Two ---

As you go to win the first prize, you discover that the claw is **nowhere near where you expected it would be**. Due to a **unit conversion error** in your measurements, the position of every prize is actually **10,000,000,000,000** higher on both the **X** and **Y** axes!

---

### Adjusted Prize Coordinates

Add **10,000,000,000,000** to the X and Y position of every prize. After making this change, the example above would now look like this:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=10000000008400, Y=10000000005400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=10000000012748, Y=10000000012176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=10000000007870, Y=10000000006450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=10000000018641, Y=10000000010279
```

---

### Updated Results

Now, it is **only possible to win a prize** on the **second** and **fourth** claw machines. Unfortunately, it will take **many more than 100 presses** to do so.

---

### Task

Using the corrected prize coordinates, **figure out how to win as many prizes as possible**.  
What is the **fewest tokens** you would have to spend to win all possible prizes?

In [3]:
# Define the updated prize locations and moves for each claw machine
offset = 10000000000000  # 0 for part 1
max_presses = offset  # 100 for part 1
arcade = Arcade.from_file("./input.txt", ClawMachine)

total_cost = 0
solutions = []
# Solve each machine
for i, machine in enumerate(arcade.machines):
    prize = [machine.prize_location[0] + offset, machine.prize_location[1] + offset]

    # analytical solution for linear system of 2 equations
    # button_a.x * n_a + button_b.x * n_b = prize.x
    # button_a.y * n_a + button_b.y * n_b = prize.x
    det = machine.move_a[0] * machine.move_b[1] - machine.move_a[1] * machine.move_b[0]
    num_a = (prize[0] * machine.move_b[1] - prize[1] * machine.move_b[0])
    num_b = (prize[1] * machine.move_a[0] - prize[0] * machine.move_a[1])
    
    if num_a % det == 0 and num_a % det == 0:  # check if solutions are integer
        cost = num_a // det * machine.cost_a + num_b // det * machine.cost_b
        total_cost += cost

total_cost # , solutions

98080815200063